<h2>Building A Chatbot </h2>
Designing and implementing an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b",groq_api_key=groq_api_key)
model

c:\Users\Nitin Mishra\OneDrive\Desktop\AIML\Gen-AI\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002100A9C8980>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002100CC3C8C0>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Nitin Mishra. Currently I am learning Langchain framework.")])

AIMessage(content='Hello Nitin! 👋 Great to meet you.\n\nIt’s exciting that you’re diving into LangChain—there’s a lot of fun (and powerful) stuff you can build with it. How’s your learning journey going so far? Are you:\n\n- Exploring the basic **chains** and **prompts**?\n- Working on a **retrieval‑augmented generation (RAG)** pipeline?\n- Trying out **agents** that can call tools or APIs?\n- Or something else entirely?\n\nIf you’d like, I can share:\n\n1. **Quick starter resources** (official docs, tutorials, and a few hands‑on notebooks).  \n2. **Common pitfalls** to watch out for when setting up LLM wrappers, vector stores, or callbacks.  \n3. **Project ideas** to cement what you’ve learned—e.g., a personal knowledge‑base chatbot, a code‑assistant, or an automated research summarizer.  \n\nJust let me know where you’re at or what you’re curious about, and I’ll tailor the next steps for you! 🚀', additional_kwargs={'reasoning_content': 'We need to respond. The user says hi, introduce

In [ ]:
# Demonstrating a hard-coded conversation history to showcase how the model can maintain context.
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Nitin Mishra. Currently I am learning Langchain framework."),
        AIMessage(content="Hello Nitin Mishra! It's nice to meet you. \n\nAs someone learning the Langchain framework, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='Your name is **Nitin\u202fMishra**, and you’re currently learning and working with the **LangChain framework**. If you’d like any help—whether it’s clarifying concepts, building a demo, or troubleshooting code—just let me know!', additional_kwargs={'reasoning_content': 'User asks: "Hey What\'s my name and what do I do?" We have prior conversation: user introduced themselves as Nitin Mishra, learning Langchain. So answer: your name is Nitin Mishra, you are learning Langchain framework. Possibly also ask if they\'d like help.'}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 160, 'total_tokens': 281, 'completion_time': 0.361758365, 'completion_tokens_details': {'reasoning_tokens': 59}, 'prompt_time': 0.006459446, 'prompt_tokens_details': None, 'queue_time': 0.054033394, 'total_time': 0.368217811}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_70d048ba3c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': N

<h3> Message History </h3>
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [14]:
config={"configurable":{"session_id":"chat1"}}

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Nitin Mishra. Currently I am learning Langchain framework.")],
    config=config
)

In [16]:
response.content

'Hey Nitin! 👋 Great to hear you’re diving into Langchain. It’s a powerful framework for building LLM‑driven applications, and there’s a lot you can do with it.\n\nIf you’d like, I can help with:\n\n* **Getting started** – Quick setup guide, key concepts, and a minimal “Hello, World” example.\n* **Core components** – How to work with **Chains**, **Agents**, **Memory**, **PromptTemplates**, and **Callbacks**.\n* **Integrations** – Connecting Langchain to vector stores (FAISS, Pinecone, Chroma), document loaders, and external APIs.\n* **Best practices** – Structuring projects, handling token limits, managing retries, and testing.\n* **Project ideas** – Small to medium‑scale projects (Q&A bots, summarizers, retrieval‑augmented generation, autonomous agents) you can build to solidify your knowledge.\n\nLet me know what you’re focusing on right now or if there’s a particular piece that’s giving you trouble. I’m happy to walk through code snippets, explain concepts, or point you to helpful re

In [ ]:
store["chat1"] 

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Nitin Mishra. Currently I am learning Langchain framework.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hey Nitin! 👋 Great to hear you’re diving into Langchain. It’s a powerful framework for building LLM‑driven applications, and there’s a lot you can do with it.\n\nIf you’d like, I can help with:\n\n* **Getting started** – Quick setup guide, key concepts, and a minimal “Hello, World” example.\n* **Core components** – How to work with **Chains**, **Agents**, **Memory**, **PromptTemplates**, and **Callbacks**.\n* **Integrations** – Connecting Langchain to vector stores (FAISS, Pinecone, Chroma), document loaders, and external APIs.\n* **Best practices** – Structuring projects, handling token limits, managing retries, and testing.\n* **Project ideas** – Small to medium‑scale projects (Q&A bots, summarizers, retrieval‑augmented generation, autonomous agents) you can build to solidify your knowle

Here is the step-by-step flow for a new session:

1. The Trigger
You call invoke with a specific session_id in the config:
```t
with_message_history.invoke(
    [HumanMessage(content="Hi...")],
    config={"configurable": {"session_id": "chat1"}}
)
```

2. The Lookup (Inside RunnableWithMessageHistory)
Before sending anything to the LLM, the RunnableWithMessageHistory wrapper pauses and looks at your config. It sees session_id="chat1".

It then calls your helper function: get_session_history("chat1").

3. Creation and Storage (Inside get_session_history)
This is where the storage happens.
```t
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        # A. Create a new empty history object
        new_history = ChatMessageHistory() 
        
        # B. Save it into the dictionary immediately
        store[session_id] = new_history 
        
    return store[session_id]
```
- Crucial Step: The moment ChatMessageHistory() is assigned to store[session_id], the dictionary now holds a reference to that history object. Any changes made to that object later will be "saved" in the dictionary because the dictionary is holding the live object.


4. The Update (Back inside RunnableWithMessageHistory)
Now that the wrapper has the history object (let's call it history_obj), it does two things:

- 1. Add User Message: It takes your input ("Hi...") and calls 
```history_obj.add_user_message("Hi...").```
    <br>Result: The object inside your store dictionary now contains [HumanMessage(content="Hi...")].
- 2. Call LLM: It sends the message to the LLM and gets a response ("Hello!").<br>
- 3. Add AI Message: It takes the response and calls ```history_obj.add_ai_message("Hello!").```<br>
Result: The object inside your store dictionary now contains ```[HumanMessage(...), AIMessage(...)].```
<h6>Summary</h6>
The store isn't updated at the end; it is updated continuously.<br>

Your function puts an empty container into the store.<br>
The Runnable wrapper keeps adding messages into that container as the conversation flows.<br>
Because store holds the

container (the object), and the wrapper keeps filling that container, the store always has the latest messages.

In [22]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is **Nitin\u202fMishra**.', additional_kwargs={'reasoning_content': 'User asks again: "What\'s my name?" Already answered: Nitin Mishra. Should respond accordingly.'}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 362, 'total_tokens': 405, 'completion_time': 0.093378925, 'completion_tokens_details': {'reasoning_tokens': 22}, 'prompt_time': 0.014964624, 'prompt_tokens_details': {'cached_tokens': 256}, 'queue_time': 0.053977056, 'total_time': 0.108343549}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_e88ce9c728', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f322c0ed-8d30-4912-8f0f-fbab4dd51589-0', usage_metadata={'input_tokens': 362, 'output_tokens': 43, 'total_tokens': 405, 'input_token_details': {'cache_read': 256}, 'output_token_details': {'reasoning': 22}})

In [25]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Ria.'

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Ria")],
    config=config1
)
response.content

'Your name is Ria.'

In [26]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Ria.'